In [0]:
import sys
workspace = dbutils.widgets.get('workspace')
sys.path.insert(0, workspace)

from libs.ingestao_silver import ingestao_silver

from pyspark.sql.functions import col, cast, trim, when, to_date, lit

In [0]:
catalog = 'projeto_cnpj'
table = 'simples'
df = spark.table(f"{catalog}.bronze.{table}")

df= (df
    .withColumn(
        'opcao_pelo_simples', 
        when(trim(col('opcao_pelo_simples')) == 'N', 'Não')
        .when(trim(col('opcao_pelo_simples')) == 'S', 'Sim')
        .otherwise('Outros')
    )
    .withColumn(
        'opcao_pelo_mei', 
        when(trim(col('opcao_pelo_mei')) == 'N', 'Não')
        .when(trim(col('opcao_pelo_mei')) == 'S', 'Sim')
        .otherwise('Outros')
    )
    .withColumn(
        'data_opcao_pelo_simples', 
        when(trim(col('data_opcao_pelo_simples')) != '00000000', to_date(col('data_opcao_pelo_simples'), 'yyyyMMdd'))
    )
    .withColumn('data_exclusao_simples', 
        when(trim(col('data_exclusao_simples')) != '00000000', to_date(col('data_exclusao_simples'), 'yyyyMMdd'))
    )
    .withColumn('data_opcao_pelo_mei', 
        when(trim(col('data_opcao_pelo_mei')) != '00000000', to_date(col('data_opcao_pelo_mei'), 'yyyyMMdd'))
    )
    .withColumn('data_exclusao_mei', 
        when(trim(col('data_exclusao_mei')) != '00000000', to_date(col('data_exclusao_mei'), 'yyyyMMdd'))
    )
    .withColumn('data_ingestao', col('data_ingestao').cast('date'))
)


In [0]:
ingestao_silver(
    spark = spark,
    df = df,
    catalog = catalog,
    table = table
)